# Efficient Stratified Inference: ESI

Conservative and exact tests and confidence bounds for the population total (or mean) of a binary population from a stratified simple random sample.

Wendell and Schmee (1996, https://www.tandfonline.com/doi/abs/10.1080/01621459.1996.10476950) proposed making inferences about the population total by maximizing a $P$-value over a set of nuisance parameters---the individual stratum totals.
They find the $P$-value by ordering possible outcomes based on the estimated population proportion: 
the test statistic is the "tail probability" of $\hat{p}$, the unbiased
estimator of the population percentage from the stratified sample.
They construct confidence bounds by inverting hypothesis tests.

Wendell and Schmee also provided R scripts for searching for the maximum over 
the allocations; the scripts became computationally impractical for more than three strata.

Maximizing the $P$-value over all allocations of $G$ ones
across $S$ strata
is combinatorially complex: Feller's "bars and stars" argument shows that there are $\binom{G+S-1}{S-1}$ ways to allocate $G$ objects among $S$ strata.
(Some of those can be ruled out, for instance if $G$ exceeds the size of any stratum.)
For $S=10$ strata of size $N_s = 400$ and $G = 300$, 
there are roughly 6.3e+16 allocations: impractical by any standard.

This document replicates the Wendell & Schmee method in python
and introduces a different conservative strategy for stratified inference, 
also based on maximizing
the $P$-value over the nuisance parameters.

The test statistic Wendell & Schmee use is sensible, but only one of infinitely many.
The new method uses a different test statistic: Fisher's combining function applied
to the tail probabilities of individual hypergeometric counts.
A naive approach to maximizing this $P$-value over the nuisance parameters 
would also involve a search over
a combinatorial number of possible allocations.
However,

1. No combinatorial search is necessary: an allocation that yields the largest
$P$-values and corresponding confidence bounds can be constructed by a simple
algorithm in order $N \log N$ operations, where $N$ is the number of items in the population. 
Runtime can be reduced further to $O( N \log S )$. 
The number of strata has little effect on the complexity of the calculation.
2. The resulting tests and confidence intervals are in some cases sharper than those of
Wendell and Schmee, in particular when the strata are heterogeneous--which is often the justification for drawing a stratified sample in the first place.

The code below implements Wright's method, the brute-force approach (enumerate all ways of allocating a given number of ones across the strata, find the $P$-value for each, and find the maximum across all allocations); the more efficient approach, which exploits special structure of the problem; and the Wendell & Schmee approach. 
It replicates a number of calculations in the Wendell & Schmee paper.
Several algorithms for finding confidence intervals are implemented, including a line 
search, a bisection-like method that takes advantage of the fact that the
possible values are integers, and a constructive algorithm for the new method.

## The problem

A population of $N$ items of which $G$ are labeled "1" and $N-G$ are labeled "0"
is partitioned into $S$ strata.
Stratum $s$ contains $N_s$ items, of which $G_s$ are labeled "1."
Thus $N = \sum_{s=1}^S N_s$ and $G := \sum_{s=1}^S G_s$.
We draw a simple random sample of size $n_s$ from stratum $s$, independently across strata.
(I.e., from stratum $s$ we draw a sample of size $n_s$ in such a way 
that every subset of $n_s$ distinct items of the $N_s$ items is equally likely;
and the $S$ samples are drawn independently.)

Let $Y_s$ denote the number of items labeled "1" in the sample from stratum $s$.
The variables $\{Y_s \}_{s=1}^S$ are independent.
The observed value of $Y_s$ is $y_s$.

We seek hypothesis tests and confidence bounds for $G$.
We first consider one-sided tests of the hypothesis $G = g$ against the 
alternative $G > g$, and
corresponding lower confidence bounds for $G$;
reversing the roles of "0" and "1" gives upper confidence 
bounds, _mutatis mutandis_.

The general strategy for testing the hypothesis $G=g$ is to 
find the largest $P$-value among all ways of allocating $g$ 
items labeled "1" among the $S$ strata 
(honoring the stratum sizes $\{N_s\}$).
That is a $P$-value for the composite hypothesis $G=g$.
The maximum can be found by examining all such allocations and
calculating the $P$-value for each.

## Wright's Method: Sum of Šidák Intervals

One easy way to get a lower confidence bound for the sum is to take the sum of
simultaneous lower confidence bounds for each stratum.
Because the samples from different strata are independent, Šidák's adjustment works.
Wright (1991) suggests this approach.

A confidence bound for $G_s$ can be constructed from $Y$ by inverting hypergeometric tests.

To have joint confidence level $1-\alpha$, make each confidence interval at $(1-\alpha)^{1/S}$.

This is an example of a much more general approach: make a joint $1-\alpha$ confidence set for all the parameters $\{G_j\}_{j=1}^S$, then find a lower bound on a functional of interest (here, their sum) over the joint set. Whenever the joint confidence set covers the parameter, the lower bound does not exceed the true value of the functional of the parameter.

## The Wendell-Schmee Test

The test statistic for the Wendell-Schmee test is the unbiased estimate
of the population proportion, $G/N$:

$$
  \hat{p} := \frac{1}{N} \sum_{s=1}^S N_s y_s/n_s.
$$
The $P$-value of the hypothesis $G_s=g_s$, $s=1, \ldots, S$, 
is the "lower tail probability" of $\hat{p}$.

Wendell and Schmee consider maximizing this lower tail probability over all allocations
of $g$ ones across strata, either by exhaustive search, or by numerical optimization
using a descent method from some number of random starting points.
(They show graphically that the tail probability is not convex in the original
parametrization.)

## Constructive maximization

For some test statistics, there is a much more efficient approach, developed here.

Define 
$$
   p_s(g_s) := \Pr \{ Y_s \ge y_s || G_s = g_s \} =
   \sum_{y = y_s}^{g_s} \frac{\binom{g_s}{y} \binom{N_s-g_s}{n_s - y}}{\binom{N_s}{n_s}},
$$
where $\binom{a}{b} := 0$ if $a \le 0$ or $b > a$.
(The double vertical bars denote "computed on the assumption that.")
This upper tail probability is a $P$-value of the hypothesis $G_s = g_s$ tested against the
alternative $G_s > g_s$.

A test of the conjunction hypothesis $G_s = g_s$, $s=1, \ldots, S$ can be constructed
using Fisher's combining function:
if all $S$ hypotheses are true, the distribution of
$$
  X^2(\vec{g}) := -2 \sum_{s=1}^S \log p_s(g_s)
$$
is dominated by the chi-square distribution with $2S$ degrees of freedom.
Let $\chi_d(z)$ denote the chance that a random variable with the chi-square 
distribution with $d$ degrees of freedom is greater than or equal to $z$.
Then a conservative $P$-value for the allocation $\vec{g}$ is
$$
   P(\vec{g}) = \chi_{2S}(X^2(\vec{g})).
$$
The allocation $\vec{g}$ of $g$ ones across strata that maximizes the $P$-value
is the allocation that minimizes $X^2(\vec{g})$ and satisfies $\sum_s g_s = g$.
Equivalently, it is the allocation that maximizes $\sum_{s=1}^S \log p_s(g_s)$.

Let 
$$
   a_s(j) := \left \{ 
                 \begin{array}{ll} 
          \log p_s(y_s), & j = y_s \\
          \log \left (p_s(j)/p_s(j-1) \right ), & j = y_s+1, \ldots N_s-(n_s-y_s).
                 \end{array}
                 \right .
$$

Then $\log p_s(g_s) = \sum_{j=y_s}^{g_s} a_s(j)$ if 
$y_s \le g_s \le N-(n_s-y_s)$, and 
$\log p_s(g_s) = -\infty$ otherwise.
Moreover, 
$$
  X^2(\vec{g}) =  -2\sum_{s=1}^S a_s(y_s) -2\sum_{s=1}^S \sum_{j=y_s+1}^{g_s} a_s(j)
$$
provided $y_s \le g_s \le N-(n_s-y_s)$, $s=1, \ldots, S$; otherwise, it is infinite.

An allocation of $g$ ones across strata is inconsistent with the data unless
$g_s \ge y_s$, $s=1, \ldots, S$.
Thus, in considering how to allocate $g$ ones to maximize the $P$-value, 
the first sum above, accounting for $\sum_s y_s$ ones, is "mandatory," or the $P$-value will be zero.
The question is how to allocate the remaining $g - \sum_s y_s$ ones to maximize
the $P$-value (equivalently, to minimize $X^2(\vec{g})$).

Let $b_k$ denote the $k$th largest element of the set 

$$
   \{a_s(j): j=y_s+1, \ldots, N_s-(n_s-y_s), \;\; s=1, \ldots, S \},
$$
with ties broken arbitrarily.
Define $\tilde{g}_y := g - \sum_{s=1}^S y_s$.

**Proposition.** For every $\vec{g}$ with $\sum_s g_s = g$, 
$$
X^2(\vec{g}) \ge X_*^2(g) := \left \{ \begin{array}{ll}
    -2 \left ( \sum_{s=1}^S a_s(y_s) + \sum_{k=1}^{\tilde{g}_y} b_k 
                \right ), & \sum_s y_s \le g \le N - \sum_s (n_s-y_s) \\
    \infty, & \mbox{ otherwise. }
    \end{array}
    \right . 
$$

**Proof.** Any $\vec{g}$ for which $X^2(\vec{g})$ is finite includes the first sum
and a sum of $\tilde{g}_y$ elements of $\{b_k\}$; the latter is at most the sum of the
$\tilde{g}_y$ largest elements of $\{b_k\}$. $\Box$

Moreover, if the $\tilde{g}_y$ largest elements of $\{b_k \}$ correspond to
an allocation of $\tilde{g}_y$ ones across the strata, the bound is sharp.
That is certainly true if $a_s(j)$ decrease monotonically with $j$ for
$j = y_s+1, \ldots, N_s-(n_s-y_s)$.
Then, if $a_s(i)$ is a term in the second sum for some $i > y_s+1$, so 
is every $a_s(j)$, $y_s \le j \le i-1$: the second sum indeed corresponds 
to a particular allocation 
$\vec{g}$ of $g$ ones across the $S$ strata, with $y_s \le g_s \le N_s-(n_s-y_s)$.
Among all allocations of $g$ items labeled "1," this one minimizes has the smallest tail 
probability, because it corresponds to the exponentiation of the smallest sum of logs 
(the largest negative sum of logs). $\Box$


**Proposition:** For $j \in y_s+1, \ldots, N_s-(n_s-y_s)$, $a_s(j)$ is monotone 
decreasing in $j$.

Let $P(g) := \max_{\vec{g}: \sum_s g_s = g} P(\vec{g})$.

**Theorem:** If $\sum_s y_s \le g \le N - \sum_s (n_s-y_s)$, 

$$
P(g) \le \chi_d(X_*^2(g)).
$$

**Proof:**
Immediate from the definitions.

The theorem shows that a "greedy" approach finds a conservative $P$-value:
Construct the values $a_s(j)$ and the set $\{b_k\}$. 
Add the $S$ values $\{a_s(x_k)$ to the $g-g_y$ largest elements of $\{b_k\}$ and 
multiply the sum by $-2$.
The upper tail probability of the chi-square distribution with $2S$ degrees of 
freedom is a conservative $P$-value for the hypothesis $G=g$.

A conservative upper $1-\alpha$ confidence bound for $G$ is the largest $g$ for which 
$P(g) \ge \alpha$.

## Changing the direction of the test

The test of the hypothesis $G=g$ given above is a one-sided test against the alternative 
$G > g$: it rejects if the chance of observing "so few" good objects is small.

To test against the alternative $G < g$ (i.e., to reject if the chance of observing "so many"
good objects is small), exchange the role of "good" and "bad."
The hypothesis $G < g$ is equivalent to the hypothesis $(N-G) > (N-g)$.

The resulting null hypothesis is $G = N-g$, and the data are $n_s - Y_s$.

## Sampling with replacement

The same approaches work for sampling with replacement.

For simplicity, assume that the population proportion $\pi = G/N$ and the stratum proportions $\pi_s G_s/N_s$ can be any numbers between $0$ and $1$, not just multiples of $1/N$ or $1/N_s$.

We observe $Y_s \sim \mathrm{Bin}(n_s, \pi_s)$, $s = 1, \ldots, S$.
The observations are independent.
The population proportion is $\pi := G/N = N^{-1}\sum_s \pi_s n_s$.

Define 
$$
   p_s(\mu_s) := \Pr \{ Y_s \ge y_s || \pi_s = \mu_s \} =
   \sum_{y = y_s}^{n_s} \binom{n_s}{y} \mu_s^y (1-\mu_s)^{n_s-y}.
$$
This is the $P$-value of the hypothesis $\pi_s = \mu_s$ tested against the
alternative $\pi_s > \mu_s$.

A test of the conjunction hypothesis $\pi_s = \mu_s$, $s=1, \ldots, S$ can be constructed
using Fisher's combining function:
Let $g_s = N_s\mu_s$, $s = 1, \ldots, S$.
If all $S$ hypotheses are true, the distribution of
$$
  X^2(\vec{g}) := -2 \sum_{s=1}^S \log p_s(\mu_s)
$$
is dominated by the chi-square distribution with $2S$ degrees of freedom.
Let $\chi_d(z)$ denote the survival function for the chi-sqare distribution with $d$ degrees
of freedom, i.e., the chance that a random variable with the chi-square 
distribution with $d$ degrees of freedom is greater than or equal to $z$.
Then a conservative $P$-value for the allocation $\vec{g}$ is
$$
   P(\vec{g}) = \chi_{2S}(X^2(\vec{\mu})).
$$
The allocation $\vec{\mu}$ that maximizes the $P$-value
is the allocation that minimizes $X^2(\vec{g})$ and satisfies $\sum_s \mu_s = g$.
Equivalently, it is the allocation that maximizes $\sum_{s=1}^S \log p_s(g_s)$.

Does a greey approach work here?

**Lemma.** The Binomial pdf is log concave in $p$.

**Proof.** 

$$
\begin{align*}
\frac{d^2}{dp^2} \log \left [ \binom{n}{k}p^k(1-p)^{n-k} \right ] & = \frac{d^2}{dp^2} \left [ C + k\log p + (n-k)\log(1-p) \right ] \\
&= \frac{d}{dp} (k/p - (n-k)/(1-p)) \\
&= -k/p^2 - (n-k)/(1-p)^2 < 0. \Box
\end{align*}
$$




## Open questions

The Wendell-Schmee test coincides exactly with the unadjusted new test (i.e., 
using the joint probability _without_ calibrating it with Fisher's combining function)
for small observed counts. The optimal parameter values
are identical, as are the $P$-values.
Why?

For what observations is the optimal allocation the same for the two tests?

Can a similar constructive/greedy
approach find the optimizer (or a bound) for the Wendell-Schmee test?
(Implausible because of the lack of convexity, at least in the original parametrization.)

What is the empirical coverage of the two methods?
What's the worst-case?

When is the new method sharper than Wendell-Schmee? (Seems to be when the strata
are heterogeneous.)

# Install permute and cryptorandom in the current kernel
import sys
!{sys.executable} -m pip install permute --user
!{sys.executable} -m pip install cryptorandom --user

In [1]:
import math
from scipy.stats import binom, hypergeom, chi2
import scipy as sp
import numpy as np
import itertools
import warnings
from permute.utils import binom_conf_interval, hypergeom_conf_interval

In [2]:
def strat_test_brute(strata, sams, found, good, alternative='upper'):
    """
    Find p-value of the hypothesis that the number G of "good" objects in a 
    stratified population is less than or equal to good, using a stratified
    random sample.
    
    Assumes that a simple random sample of size sams[s] was drawn from stratum s, 
    which contains strata[s] objects in all.
    
    The P-value is the maximum Fisher combined P-value across strata
    over all allocations of good objects among the strata. The allocations are
    enumerated using Feller's "bars and stars" construction, constrained to honor the
    stratum sizes (each stratum can contain no more "good" items than it has items in all,
    nor fewer "good" items than the sample contains).
    
    The number of allocations grows combinatorially: there can be as many as
    [(#strata + #good items) choose (#strata-1)] allocations, making the brute-force
    approach computationally infeasible when the number of strata and/or the number of
    good items is large.
    
    The test is a union-intersection test: the null hypothesis is the union over allocations
    of the intersection across strata of the hypothesis that the number of good items
    in the stratum is less than or equal to a constant.
    
    Parameters:
    -----------
    strata : list of ints
        sizes of the strata. One int per stratum.
    sams : list of ints
        the sample sizes from each stratum
    found : list of ints
        the numbers of "good" items found in the samples from the strata
    good : int
        the hypothesized total number of "good" objects in the population
    alternative : string {'lower', 'upper'}
        test against the alternative that the true value is less than good (lower)
        or greater than good (upper)
    
    Returns:
    --------
    p : float
        maximum combined p-value over all ways of allocating good "good" objects
        among the strata, honoring the stratum sizes.        
    best_part : list
        the partition that attained the maximum p-value
    """
    if alternative == 'upper': # exchange roles of "good" and "bad"
        p_func = lambda f, s, p, x: sp.stats.hypergeom.logcdf(f, s, p, x)
    elif alternative == 'lower':
        p_func = lambda f, s, p, x: sp.stats.hypergeom.logsf(f-1, s, p, x)
    else:
        raise NotImplementedError("alternative {} not implemented".format(alternative))
    best_part = found # start with what you see
    strata = np.array(strata, dtype=int)
    sams = np.array(sams, dtype=int)
    found = np.array(found, dtype=int)
    good = int(good)
    if good < np.sum(found):     
        p = 0 if alternative == 'lower' else 1 
    elif good > np.sum(strata) - np.sum(sams) + np.sum(found):
        p = 1 if alternative == 'lower' else 0
    else:  # use Feller's "bars and stars" enumeration of combinations, constrained
        log_p = np.NINF   # initial value for the max
        n_strata = len(strata)
        parts = sp.special.binom(good+n_strata-1, n_strata-1)
        if parts >= 10**7:
            print(f'warning--large number of partitions: {parts}')
        barsNstars = good + n_strata
        bars = [0]*n_strata + [barsNstars]
        partition = ([bars[j+1] - bars[j] - 1 for j in range(n_strata)] \
            for bars[1:-1] in itertools.combinations(range(1, barsNstars), n_strata-1) \
            if all(((bars[j+1] - bars[j] - 1 <= strata[j]) and \
                   (bars[j+1] - bars[j] >= found[j])) for j in range(n_strata)))
        for part in partition:
            log_p_new = 0 
            for s in range(n_strata): # should be possible to vectorize this
                log_p_new += p_func(found[s], strata[s], part[s], sams[s])
            if log_p_new > log_p:
                best_part = part
                log_p = log_p_new
        p = sp.stats.chi2.sf(-2*log_p, df = 2*len(strata))
    return p, best_part

def strat_ci_bisect(strata, sams, found, alternative='lower', cl=0.95,
                  p_value=strat_test_brute):
    """
    Confidence bound on the number of ones in a stratified population,
    based on a stratified random sample (without replacement) from
    the population.
    
    
    If alternative=='lower', finds an upper confidence bound.
    If alternative=='upper', finds a lower confidence bound.

    Uses an integer bisection search to find an exact confidence bound.
    The starting upper endpoint for the search is the unbiased estimate
    of the number of ones in the population. That could be refined in various
    ways to improve efficiency.
    
    The lower endpoint for the search is the Šidák joint lower confidence bounds,
    which should be more conservative than the exact bound.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes in the strata
    found : list of ints
        number of ones found in each stratum in each sample
    alternative : string {'lower', 'upper'}
        if alternative=='lower', finds an upper confidence bound.
        if alternative=='upper', finds a lower confidence bound.
        While this is not mnemonic, it corresponds to the sidedness of the tests
        that are inverted to get the confidence bound.
    cl : float
        confidence level. Assumed to be at least 50%.
    p_value : callable
        method for computing the p-value
    
    Returns:
    --------
    b : int
        confidence bound
    best_part : list of ints
        partition that attains the confidence bound
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'upper':  # interchange good and bad
        compl = np.array(sams)-np.array(found)  # bad items found
        cb, best_part = strat_ci_bisect(strata, sams, compl, alternative='lower', 
                                       cl=cl, p_value=p_value)
        b = np.sum(strata) - cb    # good from bad
        best_part_b = np.array(strata, dtype=int)-np.array(best_part, dtype=int)
    else:
        cl_sidak = math.pow(cl, 1/len(strata))  # Šidák adjustment
        tail = 1-cl
        a = sum((hypergeom_conf_interval( \
                sams[s], found[s], strata[s], cl=cl_sidak, alternative="lower")[0] \
                for s in range(len(strata)))) # Šidák should give a lower bound
        b = int(np.sum(np.array(strata)*np.array(found)/np.array(sams)))-1 # expected good
        p_a, best_part_a = p_value(strata, sams, found, a, alternative=alternative)
        p_b, best_part_b = p_value(strata, sams, found, b, alternative=alternative)
        tot_found = np.sum(found)
        while p_a > tail and a > tot_found:
            a = math.floor(a/2)
            p_a, best_part_a = p_value(strata, sams, found, a, alternative=alternative)
        if p_a > tail:
            b = a
            best_part_b = best_part_a
        else:
            while b-a > 1:
                c = int((a+b)/2)
                p_c, best_part_c = p_value(strata, sams, found, 
                                           c, alternative=alternative)
                if p_c > tail:
                    b, p_b, best_part_b = c, p_c, best_part_c
                elif p_c < tail:
                    a, p_a, best_part_a = c, p_c, best_part_c
                elif p_c == tail:
                    b, p_b, best_part_b = c, p_c, best_part_c
                    break
    return b, list(best_part_b)
    
def strat_test(strata, sams, found, good, alternative='lower'):
    """
    P-value for the hypothesis the number of ones in a stratified population is not 
    greater than (or not less than) a hypothesized value, based on a stratified 
    random sample without replacement from the population.
    
    Uses a fast algorithm to find (an upper bound on) the P-value constructively.
    
    Uses Fisher's combining function to combine stratum-level P-values.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes in the strata
    found : list of ints
        number of ones found in each stratum in each sample
    good : int
        hypothesized number of ones in the population
    alternative : string {'lower', 'upper'}
        test against the alternative that the true number of "good" items 
        is less than (lower) or greater than (upper) the hypothesized number, good
    
    Returns:
    --------
    p : float
        P-value
    best_part : list
        the partition that attained the maximum p-value
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'upper':                 # exchange roles of "good" and "bad"
        compl = np.array(sams) - np.array(found) # bad items found 
        bad = np.sum(strata) - good            # total bad items hypothesized
        res = strat_test(strata, sams, compl, bad, alternative='lower')
        return res[0], list(np.array(strata, dtype=int)-np.array(res[1], dtype=int))
    else:  
        good = int(good)
        best_part = []               # best partition
        if good < np.sum(found):     # impossible
            p = 0  
            best_part = found
        elif good >= np.sum(strata) - np.sum(sams) + np.sum(found): # guaranteed
            p = 1
            best_part = list(np.array(strata, dtype=int) - \
                             np.array(sams, dtype=int) + \
                             np.array(found, dtype=int))       
        elif good >= np.sum(found):  # outcome is possible under the null  
            log_p = 0                # log of joint probability
            contrib = []             # contributions to the log joint probability
            base = np.sum(found)     # must have at least this many good items
            for s in range(len(strata)):
                log_p_j = sp.stats.hypergeom.logsf(found[s]-1, strata[s], found[s], sams[s])
                                     # baseline p for minimum number of good items in stratum
                log_p += log_p_j     # log of the product of stratum-wise P-values
                for j in range(found[s]+1, strata[s]-(sams[s]-found[s])+1):
                    log_p_j1 = sp.stats.hypergeom.logsf(found[s]-1, strata[s], j, sams[s])
                                     # tail probability for j good in stratum
                    contrib.append([log_p_j1 - log_p_j, s])  
                                     # relative increase in P from new item
                    log_p_j = log_p_j1
            sorted_contrib = sorted(contrib, key = lambda x: x[0], reverse=True)
            best_part = np.array(found)
            for i in range(good-base):
                log_p += sorted_contrib[i][0]
                best_part[int(sorted_contrib[i][1])] += 1
            p = sp.stats.chi2.sf(-2*log_p, df = 2*len(strata))
    return p, list(best_part)

def strat_ci_search(strata, sams, found, alternative='lower', cl=0.95):
    """
    Confidence bound on the number of ones in a stratified population,
    based on a stratified random sample (without replacement) from
    the population.
        
    If alternative=='lower', finds an upper confidence bound.
    If alternative=='upper', finds a lower confidence bound.
    
    Searches for the allocation of items that attains the confidence bound
    by increasing the number of ones from the minimum consistent
    with the data (total found in the sample) until the P-value is greater
    than 1-cl.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes in the strata
    found : list of ints
        number of ones found in each stratum in each sample
    alternative : string {'lower', 'upper'}
        if alternative=='lower', finds an upper confidence bound.
        if alternative=='upper', finds a lower confidence bound.
        While this is not mnemonic, it corresponds to the sidedness of the tests
        that are inverted to get the confidence bound.
    cl : float
        confidence level. Assumed to be at least 50%.
    
    Returns:
    --------
    cb : int
        confidence bound
    best_part : list of ints
        partition that attains the confidence bound (give or take one item)
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'upper':  # interchange good and bad
        compl = np.array(sams)-np.array(found)  # bad items found
        cb, best_part = strat_ci(strata, sams, compl, alternative='lower', cl=cl)
        cb = np.sum(strata) - cb    # good from bad
        best_part = np.array(strata, dtype=int)-np.array(best_part, dtype=int)
    else:
        cb = int(np.sum(np.array(strata)*np.array(found)/np.array(sams)))-1 # expected good
        p_attained, best_part = strat_test(strata, sams, found, cb, 
                                                 alternative=alternative)
        while p_attained >= 1-cl:
            cb -= 1
            p_attained, best_part = strat_test(strata, sams, found, cb, 
                                                     alternative=alternative)
        cb += 1
        p_attained, best_part = strat_test(strata, sams, found, cb, 
                                                 alternative=alternative)
    return cb, list(best_part)

def strat_ci(strata, sams, found, alternative='lower', cl=0.95):
    """
    Conservative confidence bound on the number of ones in a population,
    based on a stratified random sample (without replacement) from
    the population.
    
    If alternative=='lower', finds an upper confidence bound.
    If alternative=='upper', finds a lower confidence bound.
    
    Constructs the confidence bound directly by constructing the
    allocation of the maximum number of ones that would not be
    rejected at (conservative) level 1-cl.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes in the strata
    found : list of ints
        number of ones found in each stratum in each sample
    alternative : string {'lower', 'upper'}
        if alternative=='lower', finds an upper confidence bound.
        if alternative=='upper', finds a lower confidence bound.
        While this is not mnemonic, it corresponds to the sidedness of the tests
        that are inverted to get the confidence bound.
    cl : float
        confidence level
        
    Returns:
    --------
    cb : int
        confidence bound
    best_part : list of ints
        partition that attains the confidence bound (give or take one item)
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'upper':  # interchange role of good and bad
        compl = np.array(sams)-np.array(found)  # bad found
        cb, best_part = strat_ci(strata, sams, compl, alternative='lower', cl=cl)
        best_part = np.array(strata, dtype=int)-np.array(best_part, dtype=int)
        cb = np.sum(strata) - cb   # good to bad
    else:                
        threshold = -sp.stats.chi2.ppf(cl, df=2*len(strata))/2
        # g is in the set if 
        #   chi2.sf(-2*log(p), df=2*len(strata)) >= 1-cl
        #  i.e.,   -2*log(p) <=  chi2.ppf(cl, df)
        #  i.e.,   log(p) >= -chi2.ppf(cl, df)/2
        log_p = 0                # log of joint probability
        contrib = []             # contributions to the log joint probability
        base = np.sum(found)     # must have at least this many good items
        for s in range(len(strata)):
            log_p_j = sp.stats.hypergeom.logsf(found[s]-1, strata[s], found[s], sams[s])
                                 # baseline p for minimum number of good items in stratum
            log_p += log_p_j     # log of the product of stratum-wise P-values
            small = np.PINF      # for monotonicity check
            for j in range(found[s]+1, strata[s]-(sams[s]-found[s])+1):
                log_p_j1 = sp.stats.hypergeom.logsf(found[s]-1, strata[s], j, sams[s])
                log_p_j1 = log_p_j if log_p_j1 < log_p_j else log_p_j1 # true difference is nonnegative
                contrib.append([log_p_j1 - log_p_j, s]) 
                log_p_j = log_p_j1
                if contrib[-1][0] > small:
                    print(f'reversal in stratum {s} for {j} good; old: {small} new:{contrib[-1][0]}')
                small = contrib[-1][0]
        sorted_contrib = sorted(contrib, key = lambda x: x[0], reverse=True)
        best_part = np.array(found)
        added = 0
        while log_p < threshold: 
            log_p += sorted_contrib[added][0]
            best_part[int(sorted_contrib[added][1])] += 1
            added += 1
        cb = base + added 
    return cb, list(best_part)

def strat_p(strata, sams, found, hypo, alternative='lower'):
    """
    Finds tail probability for the hypothesized population counts <hypo> for 
    simple random samples of sizes <sams> from strata of sizes <strata> if 
    <found> ones are found in the strata.
    
    Uses Fisher's combining function across strata.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes from the strata
    found : list of ints
        number of ones found in each stratum in each sample
    hypo : list of ints
        hypothesized number of ones in the strata
    
    Returns:
    --------
    p : float
        appropriate tail probability
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'lower':
        p_func = lambda x, N, G, n: sp.stats.hypergeom.sf(x-1, N, G, n)
    else:
        p_func = lambda x, N, G, n: sp.stats.hypergeom.cdf(x, N, G, n)
    p = 1    
    for s in range(len(strata)):
        p *= p_func(found[s], strata[s], hypo[s], sams[s])
    return sp.stats.chi2.sf(-2*math.log(p), df=2*len(strata))

def strat_p_ws(strata, sams, found, hypo, alternative='upper'):
    """
    Finds Wendell-Schmee P-value for the hypothesized population counts <hypo> for 
    simple random samples of sizes <sams> from strata of sizes <strata> if 
    <found> ones are found in the strata.
        
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes from the strata
    found : list of ints
        number of ones found in each stratum in each sample
    hypo : list of ints
        hypothesized number of ones in the strata
    
    Returns:
    --------
    p : float
        tail probability
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'lower':                     # exchange roles of "good" and "bad"
        compl = np.array(sams) - np.array(found)   # bad items found 
        hypo_c = np.array(strata) - np.array(hypo) # total bad items hypothesized
        p = strat_p_ws(strata, sams, compl, hypo_c, alternative='upper')
    else:    
        p_hat = lambda f, st=strata, sa=sams: np.sum(np.array(st)*np.array(f)/np.array(sa))/np.sum(st) # pooled estimate
        p_hat_0 = p_hat(found)
        per_strat = np.array(strata)/np.array(sams)/np.sum(strata)
        strat_max = np.floor(p_hat_0/per_strat)
        lo_t = (t for t in itertools.product(*[range(int(s+1)) for s in strat_max]) \
                    if p_hat(t) <= p_hat_0)
        p = sum(np.prod(sp.stats.hypergeom.pmf(t, strata, hypo, sams)) \
                    for t in lo_t)
    return p

def strat_test_ws(strata, sams, found, good, alternative='lower'):
    """
    Find p-value of the hypothesis that the number G of "good" objects in a 
    stratified population is less than or equal to good, using a stratified
    random sample.
    
    Assumes that a simple random sample of size sams[s] was drawn from stratum s, 
    which contains strata[s] objects in all.
    
    The P-value is the maximum Windell-Schmee P-value over all allocations of 
    good objects among the strata. The allocations are enumerated using Feller's 
    "bars and stars" construction, constrained to honor the stratum sizes (each 
    stratum can contain no more "good" items than it has items in all, nor fewer 
    "good" items than the sample contains).
    
    The number of allocations grows combinatorially: there can be as many as
    [(#strata + #good items) choose (#strata-1)] allocations, making the brute-force
    approach computationally infeasible when the number of strata and/or the number of
    good items is large.
    
    Parameters:
    -----------
    strata : list of ints
        sizes of the strata. One int per stratum.
    sams : list of ints
        the sample sizes from each stratum
    found : list of ints
        the numbers of "good" items found in the samples from the strata
    good : int
        the hypothesized total number of "good" objects in the population
    alternative : string {'lower', 'upper'}
        test against the alternative that the true value is less than good (lower)
        or greater than good (upper)
    
    Returns:
    --------
    p : float
        maximum combined p-value over all ways of allocating good "good" objects
        among the strata, honoring the stratum sizes.        
    best_part : list
        the partition that attained the maximum p-value
    """
    assert alternative in ['lower', 'upper']
    if alternative == 'lower':                   # exchange roles of "good" and "bad"
        compl = np.array(sams) - np.array(found) # bad items found 
        bad = np.sum(strata) - good              # total bad items hypothesized
        res = strat_test_ws(strata, sams, compl, bad, alternative='upper')
        return res[0], list(np.array(strata, dtype=int)-np.array(res[1], dtype=int))        
    best_part = found # start with what you see
    good = int(good)
    if good < np.sum(found):     
        p = 0 if alternative == 'lower' else 1 
    elif good > np.sum(strata) - np.sum(sams) + np.sum(found):
        p = 1 if alternative == 'lower' else 0
    else:  # use Feller's "bars and stars" enumeration of combinations, constrained
        p_hat = lambda f, st=strata, sa=sams: np.sum(np.array(st)*np.array(f)/np.array(sa))/np.sum(st) # pooled estimate
        p_hat_0 = p_hat(found)
        per_strat = np.array(strata)/np.array(sams)/np.sum(strata)
        strat_max = np.floor(p_hat_0/per_strat)
        p = 0   # initial value for the max
        n_strata = len(strata)
        parts = sp.special.binom(good+n_strata-1, n_strata-1)
        if parts >= 10**7:
            print(f'warning--large number of partitions: {parts}')
        barsNstars = good + n_strata
        bars = [0]*n_strata + [barsNstars]
        partition = ([bars[j+1] - bars[j] - 1 for j in range(n_strata)] \
            for bars[1:-1] in itertools.combinations(range(1, barsNstars), n_strata-1) \
            if all(((bars[j+1] - bars[j] - 1 <= strata[j]) and \
                   (bars[j+1] - bars[j] >= found[j])) for j in range(n_strata)))
        for part in partition:
            lo_t = (t for t in itertools.product(*[range(int(s+1)) for s in strat_max]) \
                    if p_hat(t) <= p_hat_0)
            p_new = 0
            for t in lo_t:
                p_temp = 1
                for s in range(len(strata)):
                    p_temp *= sp.stats.hypergeom.pmf(t[s], strata[s], part[s], sams[s])
                p_new += p_temp
            if p_new > p:
                best_part = part
                p = p_new
    return p, list(best_part)

def strat_ci_wright(strata, sams, found, alternative='lower', cl=0.95):
    """
    Confidence bound on the number of ones in a stratified population,
    based on a stratified random sample (without replacement) from
    the population.
    
    If alternative=='lower', finds an upper confidence bound.
    If alternative=='upper', finds a lower confidence bound.
    
    Constructs the confidence bound by finding Šidák multiplicity-adjusted
    joint lower confidence bounds for the number of ones in each stratum.
    
    This approach is mentioned in Wright, 1991.
    
    Parameters:
    -----------    
    strata : list of ints
        stratum sizes
    sams : list of ints
        sample sizes in the strata
    found : list of ints
        number of ones found in each stratum in each sample
    alternative : string {'lower', 'upper'}
        if alternative=='lower', finds an upper confidence bound.
        if alternative=='upper', finds a lower confidence bound.
        While this is not mnemonic, it corresponds to the sidedness of the tests
        that are inverted to get the confidence bound.
    cl : float
        confidence level
        
    Returns:
    --------
    cb : int
        confidence bound
    """
    assert alternative in ['lower', 'upper']
    inx = 0 if alternative == 'lower' else 1
    cl_sidak = math.pow(cl, 1/len(strata))  # Šidák-adjusted confidence level per stratum
    cb = sum((hypergeom_conf_interval(
                sams[s], found[s], strata[s], cl=cl_sidak, alternative=alternative)[inx] 
                for s in range(len(strata))))
    return cb

In [3]:
def test_strat_test(verbose = False):
    strata = [[10, 20, 30, 40], [20, 20, 20]]
    sams = [[2, 3, 4, 5], [5, 5, 10]]
    found = [[1, 2, 3, 4], [0, 3, 2]]
    for i in range(len(strata)):
        compl = np.array(sams[i]) - np.array(found[i])
        for j in list(range(4, 20)) + [60]:
            for alternative in ['lower', 'upper']:
                if verbose:
                    print(f'{i=} {j=} {alternative=}')
                p_exact = strat_test_brute(strata[i], sams[i], found[i], j, alternative=alternative)
                p_exact_c = strat_test_brute(strata[i], sams[i], compl, j, alternative=alternative)
                p_lkp = strat_test(strata[i], sams[i], found[i], j, alternative=alternative)
                p_lkp_c = strat_test(strata[i], sams[i], compl, j, alternative=alternative)
                np.testing.assert_allclose(p_exact[0], p_lkp[0])
                np.testing.assert_allclose(p_exact_c[0], p_lkp_c[0])

def test_strat_ci(verbose = False):
    strata = [[10, 20], [10, 20, 30, 40], [10, 20, 20, 30]]
    sams = [[5, 5], [2, 3, 4, 5], [2, 4, 5, 6]]
    found = [[2, 2], [1, 2, 3, 4], [0, 1, 2, 3]]
    for i in range(len(strata)):
        for alternative in ['lower','upper']:
            brute = strat_ci_bisect(strata[i], sams[i], found[i], alternative=alternative)
            lkp = strat_ci(strata[i], sams[i], found[i], alternative=alternative)
            lkp_s = strat_ci_search(strata[i], sams[i], found[i], alternative=alternative)
            if verbose:
                print(f'{i}-{alternative}')
                print(f'i:{i} brute:{brute[0]} lkp:{lkp[0]} lkp_s:{lkp_s[0]}\nbest_brute: {brute[1]} best_lkp:{lkp[1]} best_lkp_s: {lkp_s[1]}')
            np.testing.assert_allclose(brute[0], lkp[0])
            np.testing.assert_allclose(brute[0], lkp_s[0])
            np.testing.assert_allclose(brute[1], lkp[1])
            np.testing.assert_allclose(brute[1], lkp_s[1])


In [4]:
test_strat_test()

In [5]:
test_strat_ci()

In [6]:
strata = [[10, 20], [10, 20, 30, 40], [10, 20, 20, 30]]
sams = [[5, 5], [2, 3, 4, 5], [2, 4, 5, 6]]
found = [[2, 2], [1, 2, 3, 4], [0, 1, 2, 3]]
hypo = [[2, 2], [3, 3]]
print(strat_p(strata[0], sams[0], found[0], hypo[0], alternative='lower'))
print(strat_p(strata[0], sams[0], found[0], hypo[1], alternative='upper'))

0.06372533773032409
0.9956920718516289


## Comparison with Wendell & Schmee (1996) $P$-values and upper confidence bounds

In [7]:
def test_strat_test_ws(verbose = False):
    strata = [[10, 20, 30, 40], [20, 20, 20]]
    sams = [[2, 3, 4, 5], [5, 5, 10]]
    found = [[1, 2, 3, 4], [0, 3, 2]]
    for i in range(len(strata)):
        compl = np.array(sams[i]) - np.array(found[i])
        alternative = 'upper'
        for j in list(range(4, 20)) + [60]:
            if verbose:
                print(f'{i=} {j=} {alternative=}')
            p_exact = strat_test_ws(strata[i], sams[i], found[i], j, alternative=alternative)
            p_exact_c = strat_test_ws(strata[i], sams[i], compl, j, alternative=alternative)
            p_lkp = strat_test(strata[i], sams[i], found[i], j, alternative=alternative)
            p_lkp_c = strat_test(strata[i], sams[i], compl, j, alternative=alternative)
            print(f'ws: {p_exact[0]} ws_c: {p_exact_c[0]} best: {p_exact[1]} best_c: {p_exact_c[1]}')
            print(f'lkp: {p_lkp[0]} lkp_c: {p_lkp_c[0]} best: {p_lkp[1]} best_c: {p_lkp_c[1]}')

In [8]:
# time-consuming!
test_strat_test_ws(verbose = True)

i=0 j=4 alternative='upper'
ws: 1 ws_c: 1.0000000000000013 best: [1, 2, 3, 4] best_c: [1, 1, 2, 0]
lkp: 1 lkp_c: 1 best: [1, 2, 3, 4] best_c: [1, 1, 1, 1]
i=0 j=5 alternative='upper'
ws: 1 ws_c: 1.0000000000000027 best: [1, 2, 3, 4] best_c: [3, 0, 0, 2]
lkp: 1 lkp_c: 0.9999999988567956 best: [1, 2, 3, 4] best_c: [1, 1, 1, 2]
i=0 j=6 alternative='upper'
ws: 1 ws_c: 1.0000000000000004 best: [1, 2, 3, 4] best_c: [5, 0, 1, 0]
lkp: 1 lkp_c: 0.9999999789845687 best: [1, 2, 3, 4] best_c: [1, 1, 2, 2]
i=0 j=7 alternative='upper'
ws: 1 ws_c: 1.0000000000000024 best: [1, 2, 3, 4] best_c: [5, 0, 0, 2]
lkp: 1 lkp_c: 0.9999998660332918 best: [1, 2, 3, 4] best_c: [1, 2, 2, 2]
i=0 j=8 alternative='upper'
ws: 1 ws_c: 1.0000000000000009 best: [1, 2, 3, 4] best_c: [7, 0, 1, 0]
lkp: 1 lkp_c: 0.9999992860845038 best: [1, 2, 3, 4] best_c: [2, 2, 2, 2]
i=0 j=9 alternative='upper'
ws: 1 ws_c: 1.0000000000000029 best: [1, 2, 3, 4] best_c: [7, 0, 0, 2]
lkp: 1 lkp_c: 0.999997522402079 best: [1, 2, 3, 4] best_c:

KeyboardInterrupt: 

In [9]:
# Wendell & Schmee lead example, p. 827
strata = [100, 100]
sams = [60, 40]
found = [1, 1]
good = 10 
p = .067081
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative),\
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.067081 (0.2348557891392663, [2, 8]) (0.06708103400254271, [2, 8])


In [10]:
# Table 2, row 1
strata = [200, 100]
sams = [50, 25] 
found = [0,0]
good = 15 
p = .01194
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative), \
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.01194 (0.06481873546070695, [10, 5]) (0.011942274979969247, [10, 5])


In [11]:
# Table 2, row 2
strata = [200, 100]
sams = [50,50] 
found = [1,0]
good = 15 
p = .07232
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative), \
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.07232 (0.2622685504124782, [15, 0]) (0.07231577125487271, [15, 0])


In [12]:
# Table 2, row 3
strata = [2000, 1000]
sams = [50,50]
found = [1,0]
good = 150 
p = .09958
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative), \
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.09958 (0.32928252093496413, [150, 0]) (0.09957652360481824, [150, 0])


In [13]:
# Table 2, row 9. 
strata = [300, 200, 100]
sams = [50,50,50]
found = [1, 1, 0]
good = 30 
p = .03775
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative),\
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.03775 (0.3491101738348629, [25, 5, 0]) (0.037749475267350535, [24, 6, 0])


In [14]:
# Table 2, row 10. 
strata = [3000, 2000, 1000]
sams = [50,50,50]
found = [1, 1, 0]
good = 300 
p = .04902
alternative = 'upper'
print(p, strat_test(strata, sams, found, good, alternative=alternative),\
      strat_test_ws(strata, sams, found, good, alternative=alternative))

0.04902 (0.4006985692205336, [247, 53, 0]) (0.04902098371080246, [228, 72, 0])


In [15]:
# compare with Wendell & Schmee's R code.
# 
# > pmax.function(c(500,300,200),c(75,50,25),c(2,1,0),50)
#            [,1] [,2] [,3] [,4]
# [1,] 0.02767569   28   21    1
#
# indicating a maximum p-value of 0.02768 occurring under the null of 50 errors
# with those errors distributed as (28,21,1).
print(strat_test([500,300,200],[75,50,25],[2,1,0],50, alternative='upper'),\
      strat_test_ws([500,300,200],[75,50,25],[2,1,0],50, alternative='upper'))

(0.25567352840344126, [38, 12, 0]) (0.02767568706972813, [28, 21, 1])


In [16]:
# Table 3, Row 1
strata = [200, 100]
sams = [50,25]
found = [0, 0]
ub = 10
alternative = 'upper'
print(ub,\
      strat_ci_bisect(strata, sams, found, alternative=alternative, p_value=strat_test_ws),\
      strat_ci(strata, sams, found, alternative=alternative),\
      strat_ci_wright(strata, sams, found, alternative=alternative))

10 (10, [6, 4]) (16, [11, 5]) 23


In [ ]:
# Table 3, Row 7.
strata = [5000, 5000]
sams = [100,50]
found = [2,1] 
ub = 599
alternative = 'upper'
print(ub,\
      strat_ci_bisect(strata, sams, found, alternative=alternative, p_value=strat_test_ws),\
      strat_ci(strata, sams, found, alternative=alternative),\
      strat_ci_wright(strata, sams, found, alternative=alternative))

In [ ]:
# Table 3, Row 10. WARNING: LONG RUN TIME
strata = [3000, 2000, 1000] 
sams = [50,50,50]
found = [1, 1, 0]
ub = 298
alternative = 'upper'
print(ub,\
      strat_ci_bisect(strata, sams, found, alternative=alternative, p_value=strat_test_ws),\
      strat_ci(strata, sams, found, alternative=alternative),\
      strat_ci_wright(strata, sams, found, alternative=alternative))

In [ ]:
# Table 3, last row. LONG RUN TIME
strata = [5000, 3000, 2000]
sams = [75,50,25]
found = [2, 1, 0] 
ub = 471
alternative = 'upper'
print(ub, strat_ci(strata, sams, found, alternative=alternative), \
     strat_ci_bisect(strata, sams, found, alternative=alternative, p_value=strat_test_ws), \
     strat_ci_wright(strata, sams, found, alternative=alternative))

## When is the new method sharper than Wendell & Schmee?

In general, when there is large heterogeneity of rates across strata.


strata = [100, 100]
sams = [30,30]
found = [15,0] 
(68, [34, 34]) (68, [67, 1]) 74

strata = [100, 100]
sams = [30,30]
found = [20,0] 
(85, [43, 42]) (83, [79, 4]) 89

strata = [100, 100, 100]
sams = [25, 25, 25]
found = [20, 0, 0] 
(105, [35, 35, 35]) (102, [88, 7, 7]) 118

strata = [100, 100, 100, 100]
sams = [25, 25, 25, 25]
found = [10, 0, 0, 0] 

strata = [100, 100, 100, 100]
sams = [25,25,25,25]
found = [20,0,0,0] 
Ran for a week without completing. Fast method took under a second.
---- (107, [88, 6, 6, 7]) 131

In [ ]:
strata = [100, 100, 100, 100]
sams = [25,25,25,25]
found = [20,0,0,0] 
alternative = 'upper'
print(\
#      strat_ci_bisect(strata, sams, found, alternative=alternative, p_value=strat_test_ws),\
      strat_ci(strata, sams, found, alternative=alternative),\
      strat_ci_wright(strata, sams, found, alternative=alternative))

In [ ]:
# test empirical coverage
reps = 1000
cl = 0.95
alternative = 'upper'
strata = [5000, 3000, 2000]
sams = [75,50,25]
good = [100, 100, 500]
g = np.sum(good)
cover = 0
verb = False
for i in range(reps):
    found = sp.stats.hypergeom.rvs(strata, good, sams)
    ub = strat_ci(strata, sams, found, alternative=alternative, cl=cl)
    if verb: 
        print("f: {} ub: {} best: {}".format(found, ub[0], ub[1]))
    cover = cover+1 if ub[0] >= g else cover
    if i % 100 == 1:
        print(i+1, cover, cover/(i+1))
cover/reps